In [1]:
import  time
import  numpy as np
from    config          import *
from    agent           import Agent
from    enviroment      import Enviroment
from    IPython.display import clear_output
from    matplotlib      import pyplot as plt

In [2]:
grid_size = GRID_SIZE
num_col = grid_size

possibleActions = POSSIBLE_ACTIONS

action_space_dict = {
    "U" : 0,
    "D" : 1,
    "L" : 2,
    "R" : 3,
    "S" : 4
}
TYPE              = "random"
n_agents          = N_AGENTS
allplayerpos      = PLAYER_POS[: n_agents]
enemy_list_pos    = ENEMY_POS[: n_agents]
batch_size        = BATCH_SIZE
replay_memory_len = REPLAY_MEMORY_LEN

In [3]:
def decode_state(state_num):
    return int(state_num/num_col), state_num%num_col

def state_encode(row,col):
    return row*num_col + col 

In [4]:
def run():
    total_step = 0
    rewards_list = []
    timesteps_list = []
    total_steps = 1
    for episode in range(1, EPISODES):
        print("Episode number: ", episode)

        reward_all = 0
        time_step = 1
        i = 0

        for agent in all_agents:
            agent.terminal = False
        
        [states, enemy_states] = env.reset()
        print("enemy states: ", enemy_states)
        for agent in all_agents:
            agent.set_pos(allplayerpos[i])
            i = i + 1

        done = [False for _ in range(n_agents)]

        while not any(done):

            # env.render(clear=True)
            actions = []
            for agent in all_agents:
                actions.append(agent.act(np.concatenate((states,enemy_states)).ravel(), possibleActions))

            next_states, rewards, done = env.step(actions)
            for agent in all_agents:

                next_states = np.array(next_states)
                next_states = next_states.ravel()

                agent.set_pos(decode_state(next_states[agent.index]))
                agent.store(np.concatenate((states,enemy_states)).ravel(), action_space_dict[actions[agent.index]], \
                rewards[agent.index], np.concatenate((next_states,enemy_states)).ravel(), done[agent.index])

                if done[agent.index] == True:
                    agent.terminal = True
                    print("agent reached landmark")
                    print("updating target model")
                    agent.alighn_target_model()
            print("actions", actions)

            print("agent epsilon ", all_agents[0].epsilon, "agent memory len",\
                len(all_agents[0].expirience_replay), "steps ", time_step,\
                "reward", rewards[0], "next state ", next_states[0], "agent position ", all_agents[0].return_coordinates())

            if time_step >= TIME_STEPS:
                print("max steps reached")
                break


            total_step += 1
            time_step += 1
            total_steps+1
            states = next_states
            reward_all += sum(rewards)

            if time_step % REPLAY_STEPS == 0:
                for agent in all_agents:
                    agent.save_model()

        print("total rewards", reward_all)
        for agent in all_agents:
            agent.decay_epsilon(episode)
            agent.retrain()
        rewards_list.append(reward_all)
        timesteps_list.append(time_step)


In [5]:
all_agents= []

for index in range(n_agents):

    all_agents.append(Agent(index, allplayerpos[index]))


initial_states = []
for agent in all_agents:
    initial_states.append(state_encode(agent.x, agent.y))

enemy_states = []
for enemy_pos in enemy_list_pos:
    enemy_states.append(state_encode(enemy_pos[0], enemy_pos[1]))

env = Enviroment(initial_states = initial_states, enemy_states = enemy_states, type = "random")
env.render()

--------------------------------------------
-	-	P	-	-	

-	-	-	-	-	

-	-	-	-	-	

-	-	-	-	-	

-	-	-	-	X	

--------------------------------------------


In [12]:
run()

Episode number:  1
enemy states:  [21]
actions ['L']
agent epsilon  1.0 agent memory len 1 steps  1 reward -1 next state  1 agent position  (0, 1)
actions ['R']
agent epsilon  1.0 agent memory len 2 steps  2 reward -1 next state  2 agent position  (0, 2)
actions ['D']
agent epsilon  1.0 agent memory len 3 steps  3 reward -1 next state  7 agent position  (1, 2)
actions ['U']
agent epsilon  1.0 agent memory len 4 steps  4 reward -1 next state  2 agent position  (0, 2)
actions ['S']
agent epsilon  1.0 agent memory len 5 steps  5 reward -1 next state  2 agent position  (0, 2)
actions ['S']
agent epsilon  1.0 agent memory len 6 steps  6 reward -1 next state  2 agent position  (0, 2)
actions ['S']
agent epsilon  1.0 agent memory len 7 steps  7 reward -1 next state  2 agent position  (0, 2)
actions ['U']
agent epsilon  1.0 agent memory len 8 steps  8 reward -2 next state  2 agent position  (0, 2)
actions ['U']
agent epsilon  1.0 agent memory len 9 steps  9 reward -2 next state  2 agent positi

KeyboardInterrupt: 

In [11]:
a = [32 ,32]
b = [np.array(a).ravel()]
print(b)

[array([32, 32])]
